# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, AveragePooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Raw Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove duplicate rows
df.drop_duplicates(inplace=True)

# Slit (X,Y)
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 369)

# Oversampling: Random Oversampling

## CNN 1 AVG RO

In [3]:
# info
model_name = 'CNN 1 AVG RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97530 samples, validate on 13684 samples
Epoch 1/10000
97530/97530 [==============================] - 18s - loss: 0.5783 - acc: 0.7065 - val_loss: 0.4908 - val_acc: 0.7831
Epoch 2/10000
97530/97530 [==============================] - 12s - loss: 0.5132 - acc: 0.7561 - val_loss: 0.5798 - val_acc: 0.7100
Epoch 3/10000
97530/97530 [==============================] - 12s - loss: 0.4914 - acc: 0.7667 - val_loss: 0.4940 - val_acc: 0.7573
Epoch 4/10000
97530/97530 [==============================] - 12s - loss: 0.4765 - acc: 0.7744 - val_loss: 0.4412 - val_acc: 0.7808
Epoch 5/10000
97530/97530 [==============================] - 12s - loss: 0.4657 - acc: 0.7791 - val_loss: 0.4639 - val_acc: 0.7670
Epoch 6/10000
97530/97530 [==============================] - 12s - loss: 0.4548 - acc: 0.7852 - val_loss: 0.4511 - val_acc: 0.7800
Epoch 7/10000
97530/97530 [==============================] - 12s - loss: 0.4457 - acc: 0.7900 - val_loss: 0.5183 - val_acc: 0.7534
Epoch 8/10000
97530/97530 [=

97472/97472 [==============================] - 11s - loss: 0.4784 - acc: 0.7682 - val_loss: 0.4605 - val_acc: 0.7645
Epoch 5/10000
97472/97472 [==============================] - 11s - loss: 0.4635 - acc: 0.7760 - val_loss: 0.4905 - val_acc: 0.7525
Epoch 6/10000
97472/97472 [==============================] - 11s - loss: 0.4521 - acc: 0.7826 - val_loss: 0.5230 - val_acc: 0.7371
Epoch 7/10000
97472/97472 [==============================] - 11s - loss: 0.4422 - acc: 0.7877 - val_loss: 0.4275 - val_acc: 0.8090
Epoch 8/10000
97472/97472 [==============================] - 12s - loss: 0.4320 - acc: 0.7936 - val_loss: 0.4458 - val_acc: 0.7928
Epoch 9/10000
97472/97472 [==============================] - 12s - loss: 0.4229 - acc: 0.7991 - val_loss: 0.4446 - val_acc: 0.7852
Epoch 10/10000
97472/97472 [==============================] - 12s - loss: 0.4156 - acc: 0.8036 - val_loss: 0.4516 - val_acc: 0.7861
Epoch 11/10000
97472/97472 [==============================] - 12s - loss: 0.4066 - acc: 0.8090 -

97558/97558 [==============================] - 13s - loss: 0.3490 - acc: 0.8444 - val_loss: 0.5106 - val_acc: 0.7697
Epoch 24/10000
97558/97558 [==============================] - 13s - loss: 0.3454 - acc: 0.8450 - val_loss: 0.4446 - val_acc: 0.8132
Epoch 25/10000
97558/97558 [==============================] - 13s - loss: 0.3507 - acc: 0.8445 - val_loss: 0.4782 - val_acc: 0.7961
Epoch 26/10000
97558/97558 [==============================] - 13s - loss: 0.3432 - acc: 0.8477 - val_loss: 0.5362 - val_acc: 0.7664
Epoch 27/10000
97558/97558 [==============================] - 14s - loss: 0.3448 - acc: 0.8489 - val_loss: 0.5170 - val_acc: 0.7742
Epoch 28/10000
96992/97558 [============================>.] - ETA: 0sKF_8
Train on 97566 samples, validate on 13684 samples
Epoch 1/10000
97566/97566 [==============================] - 14s - loss: 0.5586 - acc: 0.7203 - val_loss: 0.5446 - val_acc: 0.7314
Epoch 2/10000
97566/97566 [==============================] - 14s - loss: 0.5051 - acc: 0.7596 - val_

97344/97558 [============================>.] - ETA: 0s


In [4]:
print(model_name)

metrics_kf

CNN 1 AVG RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.942679,0.722256,0.722256,0.552644,0.691915,0.557314,0.836512,00:08:15.151520
KF_2,0.918836,0.768689,0.768689,0.550002,0.723948,0.540599,0.794029,00:04:46.223834
KF_3,0.934766,0.740958,0.744280,0.548302,0.701481,0.543382,0.808998,00:05:03.709529
KF_4,0.938075,0.765937,0.765937,0.558357,0.722953,0.563723,0.831886,00:06:56.513923
KF_5,0.947307,0.741480,0.741480,0.552065,0.706394,0.552249,0.821494,00:10:08.725697
KF_6,0.948200,0.754371,0.754371,0.561026,0.724014,0.569599,0.840042,00:11:00.620702
KF_7,0.935155,0.773255,0.776578,0.545061,0.730618,0.518898,0.751776,00:06:34.107640
KF_8,0.935101,0.733688,0.737011,0.549512,0.693995,0.549044,0.822152,00:06:45.845055
KF_9,0.939179,0.749845,0.749845,0.552207,0.722478,0.548010,0.807657,00:07:15.982551
KF_10,0.947899,0.723707,0.723707,0.545736,0.702776,0.535165,0.794369,00:13:02.080019


## CNN 2 AVG RO

In [5]:
# info
model_name = 'CNN 2 AVG RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97498 samples, validate on 13684 samples
Epoch 1/10000
97498/97498 [==============================] - 15s - loss: 0.6371 - acc: 0.6648 - val_loss: 0.6361 - val_acc: 0.6793
Epoch 2/10000
97498/97498 [==============================] - 14s - loss: 0.5984 - acc: 0.6932 - val_loss: 0.5163 - val_acc: 0.6918
Epoch 3/10000
97498/97498 [==============================] - 13s - loss: 0.5540 - acc: 0.7245 - val_loss: 0.5505 - val_acc: 0.7436
Epoch 4/10000
97498/97498 [==============================] - 13s - loss: 0.5312 - acc: 0.7381 - val_loss: 0.5030 - val_acc: 0.7613
Epoch 5/10000
97498/97498 [==============================] - 13s - loss: 0.5211 - acc: 0.7439 - val_loss: 0.5799 - val_acc: 0.6935
Epoch 6/10000
97498/97498 [==============================] - 14s - loss: 0.5099 - acc: 0.7485 - val_loss: 0.5745 - val_acc: 0.6948
Epoch 7/10000
97498/97498 [==============================] - 13s - loss: 0.5011 - acc: 0.7552 - val_loss: 0.4979 - val_acc: 0.7513
Epoch 8/10000
97498/97498 [=

97384/97384 [==============================] - 13s - loss: 0.5331 - acc: 0.7429 - val_loss: 0.6219 - val_acc: 0.6933
Epoch 5/10000
97384/97384 [==============================] - 13s - loss: 0.5216 - acc: 0.7508 - val_loss: 0.4663 - val_acc: 0.7933
Epoch 6/10000
97384/97384 [==============================] - 14s - loss: 0.5122 - acc: 0.7549 - val_loss: 0.5489 - val_acc: 0.7246
Epoch 7/10000
97384/97384 [==============================] - 13s - loss: 0.5056 - acc: 0.7591 - val_loss: 0.6684 - val_acc: 0.6385
Epoch 8/10000
97384/97384 [==============================] - 13s - loss: 0.4980 - acc: 0.7614 - val_loss: 0.4823 - val_acc: 0.7537
Epoch 9/10000
97384/97384 [==============================] - 13s - loss: 0.4913 - acc: 0.7650 - val_loss: 0.6336 - val_acc: 0.6660
Epoch 10/10000
97384/97384 [==============================] - 13s - loss: 0.4874 - acc: 0.7662 - val_loss: 0.4896 - val_acc: 0.7599
Epoch 11/10000
97384/97384 [==============================] - 13s - loss: 0.4806 - acc: 0.7691 -

97426/97426 [==============================] - 12s - loss: 0.3729 - acc: 0.8304 - val_loss: 0.5148 - val_acc: 0.7549
Epoch 48/10000
97426/97426 [==============================] - 12s - loss: 0.3703 - acc: 0.8331 - val_loss: 0.4756 - val_acc: 0.7694
Epoch 49/10000
97426/97426 [==============================] - 12s - loss: 0.3707 - acc: 0.8331 - val_loss: 0.3892 - val_acc: 0.8160
Epoch 50/10000
97312/97426 [============================>.] - ETA: 0sKF_7
Train on 97550 samples, validate on 13684 samples
Epoch 1/10000
97550/97550 [==============================] - 13s - loss: 0.6279 - acc: 0.6705 - val_loss: 0.5453 - val_acc: 0.7134
Epoch 2/10000
97550/97550 [==============================] - 12s - loss: 0.5749 - acc: 0.7100 - val_loss: 0.5421 - val_acc: 0.7342
Epoch 3/10000
97550/97550 [==============================] - 12s - loss: 0.5330 - acc: 0.7407 - val_loss: 0.6346 - val_acc: 0.6233
Epoch 4/10000
97550/97550 [==============================] - 12s - loss: 0.5181 - acc: 0.7501 - val_lo

97594/97594 [==============================] - 12s - loss: 0.4101 - acc: 0.8070 - val_loss: 0.4875 - val_acc: 0.7295
Epoch 32/10000
97594/97594 [==============================] - 12s - loss: 0.4071 - acc: 0.8070 - val_loss: 0.4400 - val_acc: 0.7822
Epoch 33/10000
97594/97594 [==============================] - 12s - loss: 0.4019 - acc: 0.8106 - val_loss: 0.3985 - val_acc: 0.7898
Epoch 34/10000
97594/97594 [==============================] - 12s - loss: 0.4023 - acc: 0.8110 - val_loss: 0.4321 - val_acc: 0.7935
Epoch 35/10000
97594/97594 [==============================] - 12s - loss: 0.4114 - acc: 0.8059 - val_loss: 0.4858 - val_acc: 0.7245
Epoch 36/10000
97594/97594 [==============================] - 12s - loss: 0.4081 - acc: 0.8094 - val_loss: 0.4640 - val_acc: 0.7653
Epoch 37/10000
97594/97594 [==============================] - 12s - loss: 0.4003 - acc: 0.8116 - val_loss: 0.4305 - val_acc: 0.7905
Epoch 38/10000
97594/97594 [==============================] - 12s - loss: 0.3958 - acc: 0.8

In [6]:
print(model_name)

metrics_kf

CNN 2 AVG RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.920394,0.750999,0.750999,0.545803,0.720407,0.527396,0.771932,00:10:42.134184
KF_2,0.907358,0.803176,0.806498,0.542112,0.736689,0.499265,0.711430,00:09:39.266594
KF_3,0.897030,0.791828,0.795151,0.543429,0.737420,0.506376,0.725072,00:06:42.577426
KF_4,0.886198,0.816518,0.816518,0.539195,0.738902,0.474259,0.660616,00:05:34.093082
KF_5,0.923107,0.760526,0.760526,0.545249,0.711623,0.529449,0.779532,00:11:32.717395
KF_6,0.924375,0.777868,0.781190,0.549141,0.731282,0.534738,0.780584,00:10:40.561921
KF_7,0.904300,0.803239,0.803239,0.553938,0.742412,0.546278,0.795843,00:07:51.290273
KF_8,0.918292,0.766098,0.766098,0.549396,0.723815,0.538671,0.790713,00:10:14.885929
KF_9,0.915102,0.777089,0.777089,0.548825,0.724649,0.536669,0.787265,00:09:52.687819
KF_10,0.910384,0.755990,0.755990,0.543921,0.718875,0.520878,0.760821,00:08:39.923007


## CNN 3 AVG RO

In [7]:
# info
model_name = 'CNN 3 AVG RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97580 samples, validate on 13684 samples
Epoch 1/10000
97580/97580 [==============================] - 14s - loss: 0.6884 - acc: 0.5465 - val_loss: 0.6604 - val_acc: 0.7026
Epoch 2/10000
97580/97580 [==============================] - 13s - loss: 0.6523 - acc: 0.6368 - val_loss: 0.6086 - val_acc: 0.7214
Epoch 3/10000
97580/97580 [==============================] - 12s - loss: 0.6458 - acc: 0.6603 - val_loss: 0.6268 - val_acc: 0.7242
Epoch 4/10000
97580/97580 [==============================] - 12s - loss: 0.6412 - acc: 0.6657 - val_loss: 0.6211 - val_acc: 0.7284
Epoch 5/10000
97580/97580 [==============================] - 12s - loss: 0.6356 - acc: 0.6681 - val_loss: 0.5508 - val_acc: 0.7318
Epoch 6/10000
97580/97580 [==============================] - 12s - loss: 0.6191 - acc: 0.6741 - val_loss: 0.6317 - val_acc: 0.6835
Epoch 7/10000
97580/97580 [==============================] - 12s - loss: 0.5905 - acc: 0.7031 - val_loss: 0.7080 - val_acc: 0.5727
Epoch 8/10000
97580/97580 [=

97444/97444 [==============================] - 13s - loss: 0.5579 - acc: 0.7266 - val_loss: 0.3814 - val_acc: 0.8438
Epoch 18/10000
97444/97444 [==============================] - 13s - loss: 0.5547 - acc: 0.7289 - val_loss: 0.4975 - val_acc: 0.7312
Epoch 19/10000
97444/97444 [==============================] - 13s - loss: 0.5553 - acc: 0.7270 - val_loss: 0.4338 - val_acc: 0.7796
Epoch 20/10000
97444/97444 [==============================] - 12s - loss: 0.5548 - acc: 0.7269 - val_loss: 0.6279 - val_acc: 0.5906
Epoch 21/10000
97444/97444 [==============================] - 12s - loss: 0.5524 - acc: 0.7293 - val_loss: 0.5970 - val_acc: 0.6103
Epoch 22/10000
97444/97444 [==============================] - 12s - loss: 0.5519 - acc: 0.7288 - val_loss: 0.4871 - val_acc: 0.7311
Epoch 23/10000
97444/97444 [==============================] - 12s - loss: 0.5496 - acc: 0.7309 - val_loss: 0.5902 - val_acc: 0.6158
Epoch 24/10000
97444/97444 [==============================] - 13s - loss: 0.5482 - acc: 0.7

97398/97398 [==============================] - 13s - loss: 0.5502 - acc: 0.7280 - val_loss: 0.4972 - val_acc: 0.7017
Epoch 38/10000
97398/97398 [==============================] - 13s - loss: 0.5503 - acc: 0.7278 - val_loss: 0.4172 - val_acc: 0.7865
Epoch 39/10000
97398/97398 [==============================] - 13s - loss: 0.5472 - acc: 0.7303 - val_loss: 0.5514 - val_acc: 0.6343
Epoch 40/10000
97398/97398 [==============================] - 13s - loss: 0.5495 - acc: 0.7297 - val_loss: 0.5170 - val_acc: 0.7005
Epoch 41/10000
97398/97398 [==============================] - 13s - loss: 0.5487 - acc: 0.7299 - val_loss: 0.4431 - val_acc: 0.7563
Epoch 42/10000
97398/97398 [==============================] - 12s - loss: 0.5483 - acc: 0.7291 - val_loss: 0.5685 - val_acc: 0.6369
Epoch 43/10000
97398/97398 [==============================] - 13s - loss: 0.5472 - acc: 0.7307 - val_loss: 0.5502 - val_acc: 0.6835
Epoch 44/10000
97398/97398 [==============================] - 13s - loss: 0.5465 - acc: 0.7

97590/97590 [==============================] - 13s - loss: 0.5409 - acc: 0.7357 - val_loss: 0.5203 - val_acc: 0.6910
Epoch 29/10000
97590/97590 [==============================] - 13s - loss: 0.5376 - acc: 0.7379 - val_loss: 0.5680 - val_acc: 0.6582
Epoch 30/10000
97590/97590 [==============================] - 10s    
KF_10
Train on 97562 samples, validate on 13684 samples
Epoch 1/10000
97562/97562 [==============================] - 14s - loss: 0.6900 - acc: 0.5347 - val_loss: 0.6731 - val_acc: 0.6763
Epoch 2/10000
97562/97562 [==============================] - 12s - loss: 0.6495 - acc: 0.6495 - val_loss: 0.6481 - val_acc: 0.7014
Epoch 3/10000
97562/97562 [==============================] - 12s - loss: 0.6391 - acc: 0.6556 - val_loss: 0.5564 - val_acc: 0.7170
Epoch 4/10000
97562/97562 [==============================] - 12s - loss: 0.6346 - acc: 0.6606 - val_loss: 0.5724 - val_acc: 0.7024
Epoch 5/10000
97562/97562 [==============================] - 13s - loss: 0.6286 - acc: 0.6649 - val_l

In [8]:
print(model_name)

metrics_kf

CNN 3 AVG RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.836387,0.804314,0.807636,0.546146,0.739593,0.518573,0.747600,00:11:04.719896
KF_2,0.826349,0.829491,0.829491,0.558940,0.767069,0.553178,0.797317,00:12:28.345105
KF_3,0.822036,0.806074,0.806074,0.540622,0.717160,0.505894,0.732044,00:07:22.176702
KF_4,0.829828,0.808432,0.808432,0.547310,0.744267,0.521093,0.750460,00:08:30.238576
KF_5,0.827878,0.802811,0.802811,0.538362,0.720761,0.488706,0.696133,00:06:51.177134
KF_6,0.826348,0.810717,0.810717,0.539880,0.733692,0.486640,0.687319,00:13:07.893014
KF_7,0.825530,0.805012,0.808334,0.560355,0.739257,0.564787,0.826493,00:08:14.198773
KF_8,0.827179,0.797145,0.797145,0.536216,0.727363,0.458524,0.632334,00:07:30.359539
KF_9,0.829044,0.801797,0.801797,0.540773,0.725727,0.500657,0.718721,00:06:45.840898
KF_10,0.832827,0.807467,0.807467,0.541733,0.740249,0.494217,0.700566,00:08:34.038303


# Oversampling: SMOTE

## CNN 1 AVG SM

In [9]:
# info
model_name = 'CNN 1 AVG SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97595 samples, validate on 13684 samples
Epoch 1/10000
97595/97595 [==============================] - 12s - loss: 0.4370 - acc: 0.8106 - val_loss: 0.3357 - val_acc: 0.8636
Epoch 2/10000
97595/97595 [==============================] - 12s - loss: 0.3483 - acc: 0.8565 - val_loss: 0.4214 - val_acc: 0.8362
Epoch 3/10000
97595/97595 [==============================] - 12s - loss: 0.3232 - acc: 0.8665 - val_loss: 0.3298 - val_acc: 0.8508
Epoch 4/10000
97595/97595 [==============================] - 12s - loss: 0.3068 - acc: 0.8756 - val_loss: 0.2902 - val_acc: 0.8696
Epoch 5/10000
97595/97595 [==============================] - 12s - loss: 0.2961 - acc: 0.8807 - val_loss: 0.3639 - val_acc: 0.8510
Epoch 6/10000
97595/97595 [==============================] - 12s - loss: 0.2855 - acc: 0.8848 - val_loss: 0.2532 - val_acc: 0.8851
Epoch 7/10000
97595/97595 [==============================] - 12s - loss: 0.2799 - acc: 0.8875 - val_loss: 0.3518 - val_acc: 0.8438
Epoch 8/10000
97595/97595 [=

97528/97528 [==============================] - 12s - loss: 0.2521 - acc: 0.8996 - val_loss: 0.3508 - val_acc: 0.8471
Epoch 12/10000
97528/97528 [==============================] - 12s - loss: 0.2483 - acc: 0.9005 - val_loss: 0.4099 - val_acc: 0.8240
Epoch 13/10000
97528/97528 [==============================] - 12s - loss: 0.2461 - acc: 0.9028 - val_loss: 0.2998 - val_acc: 0.8761
Epoch 14/10000
97528/97528 [==============================] - 12s - loss: 0.2442 - acc: 0.9034 - val_loss: 0.3400 - val_acc: 0.8532
Epoch 15/10000
97528/97528 [==============================] - 12s - loss: 0.2400 - acc: 0.9058 - val_loss: 0.2833 - val_acc: 0.8785
Epoch 16/10000
97528/97528 [==============================] - 12s - loss: 0.2376 - acc: 0.9067 - val_loss: 0.2630 - val_acc: 0.8857
Epoch 17/10000
97528/97528 [==============================] - 12s - loss: 0.2363 - acc: 0.9068 - val_loss: 0.2778 - val_acc: 0.8815
Epoch 18/10000
97528/97528 [==============================] - 12s - loss: 0.2325 - acc: 0.9

97676/97676 [==============================] - 12s - loss: 0.2437 - acc: 0.9031 - val_loss: 0.3339 - val_acc: 0.8620
Epoch 16/10000
97676/97676 [==============================] - 12s - loss: 0.2397 - acc: 0.9045 - val_loss: 0.2215 - val_acc: 0.9157
Epoch 17/10000
97676/97676 [==============================] - 12s - loss: 0.2390 - acc: 0.9046 - val_loss: 0.2929 - val_acc: 0.8701
Epoch 18/10000
97676/97676 [==============================] - 12s - loss: 0.2358 - acc: 0.9068 - val_loss: 0.2806 - val_acc: 0.8818
Epoch 19/10000
97676/97676 [==============================] - 12s - loss: 0.2353 - acc: 0.9071 - val_loss: 0.6288 - val_acc: 0.7181
Epoch 20/10000
97676/97676 [==============================] - 12s - loss: 0.2326 - acc: 0.9081 - val_loss: 0.4484 - val_acc: 0.8156
Epoch 21/10000
97676/97676 [==============================] - 12s - loss: 0.2307 - acc: 0.9092 - val_loss: 0.3332 - val_acc: 0.8391
Epoch 22/10000
97676/97676 [==============================] - 12s - loss: 0.2300 - acc: 0.9

In [10]:
print(model_name)

metrics_kf

CNN 1 AVG SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.969579,0.772999,0.772999,0.563557,0.667690,0.580829,0.881757,00:05:43.813613
KF_2,0.971524,0.772956,0.772956,0.558153,0.704397,0.567041,0.845193,00:09:04.253122
KF_3,0.973707,0.776853,0.776853,0.575418,0.697566,0.597716,0.887135,00:08:19.962591
KF_4,0.970824,0.780646,0.780646,0.575397,0.700135,0.597652,0.885951,00:05:04.279289
KF_5,0.974497,0.765016,0.765016,0.561566,0.698700,0.574795,0.858721,00:08:56.945047
KF_6,0.971102,0.782594,0.782594,0.562952,0.720365,0.574357,0.848329,00:05:07.772463
KF_7,0.971344,0.770965,0.774287,0.567483,0.715294,0.583816,0.863062,00:05:14.080108
KF_8,0.971476,0.750439,0.750439,0.559928,0.683989,0.573556,0.864115,00:07:39.072087
KF_9,0.968903,0.788331,0.788331,0.567771,0.711469,0.584808,0.866333,00:04:53.608857
KF_10,0.970461,0.741672,0.741672,0.563250,0.697520,0.578145,0.864097,00:05:43.576128


## CNN 2 AVG SM

In [11]:
# info
model_name = 'CNN 2 AVG SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97503 samples, validate on 13684 samples
Epoch 1/10000
97503/97503 [==============================] - 13s - loss: 0.5517 - acc: 0.7491 - val_loss: 0.5510 - val_acc: 0.6964
Epoch 2/10000
97503/97503 [==============================] - 12s - loss: 0.4128 - acc: 0.8263 - val_loss: 0.4783 - val_acc: 0.7835
Epoch 3/10000
97503/97503 [==============================] - 12s - loss: 0.3676 - acc: 0.8475 - val_loss: 0.3820 - val_acc: 0.8430
Epoch 4/10000
97503/97503 [==============================] - 12s - loss: 0.3464 - acc: 0.8577 - val_loss: 0.4626 - val_acc: 0.8058
Epoch 5/10000
97503/97503 [==============================] - 12s - loss: 0.3338 - acc: 0.8623 - val_loss: 0.3501 - val_acc: 0.8419
Epoch 6/10000
97503/97503 [==============================] - 12s - loss: 0.3245 - acc: 0.8666 - val_loss: 0.3281 - val_acc: 0.8559
Epoch 7/10000
97503/97503 [==============================] - 12s - loss: 0.3168 - acc: 0.8694 - val_loss: 0.3013 - val_acc: 0.8646
Epoch 8/10000
97503/97503 [=

97600/97706 [============================>.] - ETA: 0sKF_4
Train on 97548 samples, validate on 13684 samples
Epoch 1/10000
97548/97548 [==============================] - 12s - loss: 0.5697 - acc: 0.7389 - val_loss: 0.4661 - val_acc: 0.7249
Epoch 2/10000
97548/97548 [==============================] - 12s - loss: 0.4762 - acc: 0.7875 - val_loss: 0.3821 - val_acc: 0.7617
Epoch 3/10000
97548/97548 [==============================] - 12s - loss: 0.3902 - acc: 0.8366 - val_loss: 0.2279 - val_acc: 0.9038
Epoch 4/10000
97548/97548 [==============================] - 12s - loss: 0.3550 - acc: 0.8549 - val_loss: 0.1670 - val_acc: 0.9421
Epoch 5/10000
97548/97548 [==============================] - 12s - loss: 0.3381 - acc: 0.8619 - val_loss: 0.7487 - val_acc: 0.6505
Epoch 6/10000
97548/97548 [==============================] - 12s - loss: 0.3225 - acc: 0.8682 - val_loss: 0.5696 - val_acc: 0.7322
Epoch 7/10000
97548/97548 [==============================] - 12s - loss: 0.3151 - acc: 0.8734 - val_loss:

97490/97490 [==============================] - 12s - loss: 0.2727 - acc: 0.8892 - val_loss: 0.3210 - val_acc: 0.8475
Epoch 21/10000
97490/97490 [==============================] - 13s - loss: 0.2704 - acc: 0.8915 - val_loss: 0.2825 - val_acc: 0.8740
Epoch 22/10000
97490/97490 [==============================] - 13s - loss: 0.2699 - acc: 0.8912 - val_loss: 0.3685 - val_acc: 0.8271
Epoch 23/10000
97490/97490 [==============================] - 12s - loss: 0.2672 - acc: 0.8930 - val_loss: 0.2252 - val_acc: 0.9024
Epoch 24/10000
97490/97490 [==============================] - 12s - loss: 0.2650 - acc: 0.8936 - val_loss: 0.2891 - val_acc: 0.8756
Epoch 25/10000
97490/97490 [==============================] - 12s - loss: 0.2642 - acc: 0.8946 - val_loss: 0.3611 - val_acc: 0.8436
Epoch 26/10000
97490/97490 [==============================] - 12s - loss: 0.2639 - acc: 0.8946 - val_loss: 0.3680 - val_acc: 0.8313
Epoch 27/10000
97490/97490 [==============================] - 12s - loss: 0.2626 - acc: 0.8

97440/97440 [==============================] - 13s - loss: 0.2817 - acc: 0.8874 - val_loss: 0.2507 - val_acc: 0.8878
Epoch 21/10000
97440/97440 [==============================] - 12s - loss: 0.2800 - acc: 0.8883 - val_loss: 0.2544 - val_acc: 0.8762
Epoch 22/10000
97440/97440 [==============================] - 12s - loss: 0.2764 - acc: 0.8891 - val_loss: 0.4997 - val_acc: 0.7767
Epoch 23/10000
97440/97440 [==============================] - 12s - loss: 0.2768 - acc: 0.8895 - val_loss: 0.3693 - val_acc: 0.8399
Epoch 24/10000
97440/97440 [==============================] - 12s - loss: 0.2722 - acc: 0.8916 - val_loss: 0.3477 - val_acc: 0.8480
Epoch 25/10000
97440/97440 [==============================] - 12s - loss: 0.2722 - acc: 0.8916 - val_loss: 0.3565 - val_acc: 0.8403
Epoch 26/10000
97440/97440 [==============================] - 12s - loss: 0.2703 - acc: 0.8921 - val_loss: 0.3361 - val_acc: 0.8482
Epoch 27/10000
97440/97440 [==============================] - 12s - loss: 0.2692 - acc: 0.8

In [12]:
print(model_name)

metrics_kf

CNN 2 AVG SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.963028,0.781001,0.781001,0.563724,0.695967,0.579089,0.865711,00:06:52.706949
KF_2,0.957992,0.807656,0.807656,0.591974,0.702724,0.619466,0.906221,00:06:42.319720
KF_3,0.968054,0.763879,0.763879,0.562695,0.698614,0.576923,0.861615,00:11:52.490762
KF_4,0.953588,0.793683,0.793683,0.564400,0.713531,0.578256,0.856617,00:05:11.736301
KF_5,0.962199,0.790611,0.793933,0.560100,0.718259,0.568664,0.841226,00:05:26.223925
KF_6,0.969369,0.783984,0.787306,0.558531,0.714047,0.566014,0.839253,00:10:41.668209
KF_7,0.964664,0.793047,0.796370,0.579491,0.692256,0.603302,0.895291,00:09:29.692292
KF_8,0.956753,0.778845,0.778845,0.547153,0.697474,0.541367,0.807419,00:05:42.494145
KF_9,0.963012,0.778107,0.781440,0.544661,0.718760,0.523998,0.766741,00:10:27.484403
KF_10,0.965956,0.776466,0.779799,0.564407,0.714393,0.578251,0.856598,00:11:01.182314


## CNN 3 AVG SM

In [13]:
# info
model_name = 'CNN 3 AVG SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97535 samples, validate on 13684 samples
Epoch 1/10000
97535/97535 [==============================] - 14s - loss: 0.6893 - acc: 0.5444 - val_loss: 0.6997 - val_acc: 0.2002
Epoch 2/10000
97535/97535 [==============================] - 13s - loss: 0.6261 - acc: 0.6778 - val_loss: 0.5729 - val_acc: 0.7177
Epoch 3/10000
97535/97535 [==============================] - 12s - loss: 0.5696 - acc: 0.7323 - val_loss: 0.5901 - val_acc: 0.7080
Epoch 4/10000
97535/97535 [==============================] - 13s - loss: 0.5455 - acc: 0.7516 - val_loss: 0.5345 - val_acc: 0.7084
Epoch 5/10000
97535/97535 [==============================] - 12s - loss: 0.5278 - acc: 0.7562 - val_loss: 0.5069 - val_acc: 0.7080
Epoch 6/10000
97535/97535 [==============================] - 13s - loss: 0.5103 - acc: 0.7629 - val_loss: 0.5115 - val_acc: 0.6595
Epoch 7/10000
97535/97535 [==============================] - 12s - loss: 0.4980 - acc: 0.7658 - val_loss: 0.5110 - val_acc: 0.7047
Epoch 8/10000
97535/97535 [=

97599/97599 [==============================] - 13s - loss: 0.3921 - acc: 0.8350 - val_loss: 0.3597 - val_acc: 0.8292
Epoch 20/10000
97599/97599 [==============================] - 12s - loss: 0.3890 - acc: 0.8359 - val_loss: 0.4111 - val_acc: 0.8251
Epoch 21/10000
97599/97599 [==============================] - 13s - loss: 0.3882 - acc: 0.8375 - val_loss: 0.3414 - val_acc: 0.8461
Epoch 22/10000
97599/97599 [==============================] - 12s - loss: 0.3873 - acc: 0.8372 - val_loss: 0.4737 - val_acc: 0.7594
Epoch 23/10000
97599/97599 [==============================] - 13s - loss: 0.3853 - acc: 0.8373 - val_loss: 0.3934 - val_acc: 0.8129
Epoch 24/10000
97599/97599 [==============================] - 12s - loss: 0.3818 - acc: 0.8395 - val_loss: 0.3448 - val_acc: 0.8376
Epoch 25/10000
97599/97599 [==============================] - 12s - loss: 0.3829 - acc: 0.8381 - val_loss: 0.2615 - val_acc: 0.8680
Epoch 26/10000
97599/97599 [==============================] - 14s - loss: 0.3816 - acc: 0.8

97448/97448 [==============================] - 13s - loss: 0.4133 - acc: 0.8247 - val_loss: 0.3885 - val_acc: 0.8482
Epoch 15/10000
97448/97448 [==============================] - 12s - loss: 0.4091 - acc: 0.8261 - val_loss: 0.4051 - val_acc: 0.8385
Epoch 16/10000
97448/97448 [==============================] - 12s - loss: 0.4076 - acc: 0.8267 - val_loss: 0.3446 - val_acc: 0.8670
Epoch 17/10000
97448/97448 [==============================] - 12s - loss: 0.4043 - acc: 0.8275 - val_loss: 0.3579 - val_acc: 0.8704
Epoch 18/10000
97448/97448 [==============================] - 13s - loss: 0.4005 - acc: 0.8287 - val_loss: 0.3260 - val_acc: 0.8809
Epoch 19/10000
97448/97448 [==============================] - 12s - loss: 0.3944 - acc: 0.8336 - val_loss: 0.4170 - val_acc: 0.8524
Epoch 20/10000
97448/97448 [==============================] - 12s - loss: 0.3918 - acc: 0.8353 - val_loss: 0.3942 - val_acc: 0.8508
Epoch 21/10000
97448/97448 [==============================] - 13s - loss: 0.3896 - acc: 0.8

97573/97573 [==============================] - 13s - loss: 0.3612 - acc: 0.8491 - val_loss: 0.2986 - val_acc: 0.8818
Epoch 51/10000
97573/97573 [==============================] - 13s - loss: 0.3604 - acc: 0.8495 - val_loss: 0.4171 - val_acc: 0.8347
Epoch 52/10000
97573/97573 [==============================] - 13s - loss: 0.3600 - acc: 0.8499 - val_loss: 0.4167 - val_acc: 0.8344
Epoch 53/10000
97573/97573 [==============================] - 13s - loss: 0.3586 - acc: 0.8500 - val_loss: 0.2870 - val_acc: 0.8723
Epoch 54/10000
97573/97573 [==============================] - 13s - loss: 0.3561 - acc: 0.8512 - val_loss: 0.3393 - val_acc: 0.8574
Epoch 55/10000
97573/97573 [==============================] - 13s - loss: 0.3603 - acc: 0.8498 - val_loss: 0.3792 - val_acc: 0.8365
Epoch 56/10000
97573/97573 [==============================] - 13s - loss: 0.3576 - acc: 0.8502 - val_loss: 0.3340 - val_acc: 0.8652
Epoch 57/10000
97573/97573 [==============================] - 13s - loss: 0.3592 - acc: 0.8

In [14]:
print(model_name)

metrics_kf

CNN 3 AVG SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.934375,0.792239,0.792239,0.555371,0.694225,0.562957,0.844009,00:09:54.630364
KF_2,0.934989,0.806272,0.809595,0.574015,0.733557,0.593685,0.867552,00:13:52.769989
KF_3,0.934371,0.776475,0.779798,0.547546,0.703245,0.540518,0.803210,00:08:22.115854
KF_4,0.935838,0.800839,0.804161,0.565055,0.727420,0.577548,0.849645,00:12:06.347426
KF_5,0.926009,0.779462,0.779462,0.566285,0.699469,0.583239,0.869376,00:08:33.367389
KF_6,0.935697,0.795733,0.799055,0.552746,0.737155,0.544429,0.794922,00:09:13.782251
KF_7,0.928057,0.801952,0.805274,0.569790,0.737147,0.585543,0.856091,00:09:45.451206
KF_8,0.937648,0.774006,0.774006,0.565707,0.682942,0.583462,0.877401,00:13:19.944048
KF_9,0.926650,0.789813,0.789813,0.557044,0.708481,0.563931,0.839100,00:09:07.617916
KF_10,0.884968,0.726705,0.726705,0.531851,0.692169,0.467777,0.663465,00:10:46.012744


# Metrics

In [15]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
CNN 1 AVG RO,0.938720,0.747419,0.748415,0.551491,0.712057,0.547798,0.810891,00:07:58.896047
CNN 2 AVG RO,0.910654,0.780333,0.781330,0.546101,0.728607,0.521398,0.756381,00:09:09.013763
CNN 3 AVG RO,0.828341,0.807326,0.807990,0.545034,0.735514,0.509227,0.728899,00:09:02.898794
CNN 1 AVG SM,0.971342,0.770247,0.770579,0.565547,0.699713,0.581272,0.866469,00:06:34.736330
CNN 2 AVG SM,0.962461,0.784728,0.786391,0.563714,0.706602,0.573533,0.849669,00:08:20.799902
CNN 3 AVG SM,0.927860,0.784350,0.786011,0.558541,0.711581,0.560309,0.826477,00:10:30.203918
